## Imports 

In [1]:
from statistics import mean
import numpy as np
import pandas as pd
import math
import os
from collections import Counter
from functools import reduce
import glob
import copy

## Opening the CSV files 

In [2]:
dataframes = [pd.read_csv(file, sep=',', index_col=0) for file in sorted(glob.glob('../preprocessed_datasets' + "/*."+'csv'))]
cohorts = [file.strip(".csv") for file in sorted(os.listdir('../preprocessed_datasets'))]

In [3]:
# reduce to BL visit only
all_cohorts = dict()
for name, df in zip(cohorts, dataframes):
    all_cohorts[name] = df.loc[(df["Visit"] == 1) & (df["Diagnosis"].astype(str) == 'CU')]

## Functions to perform essential calculations 

In [4]:
def cat_stat_df(dfs, result):
    """Counting different categories, calculate the % of categorical features, store results in a df"""
    
    categorical = {'APOE4': [2.0, 1.0], 'Sex': ['Female'], 'Diagnosis': ['CU', 'MCI', 'AD']}
    column_cat = ['Sex', 'Diagnosis', 'APOE4']

    for cohort in dfs:
        
        if dfs[cohort].empty==True:
            continue
        
        else:
            calc_dict = dict()
            df = dfs[cohort]

            for col in column_cat:
                ca = Counter(df[col].dropna())
                calc_dict[col] = ca

            cohort_df = pd.DataFrame(calc_dict).transpose()
            cohort_df = cohort_df.dropna(how='all')
            cohort_df.loc[cohort] = cohort_df.sum()
           
            for i in categorical:
                
                if i == 'Diagnosis':
                    
                    if i in cohort_df.index: 
                        result.loc[cohort, categorical[i]] = cohort_df.loc[cohort, cohort_df.loc[i].notna()].astype(int)
                        result.loc[cohort, categorical[i]] = result.loc[cohort, categorical[i]].replace({np.nan: 0})
                        result.loc[cohort, 'n'] = int(sum(cohort_df.loc[cohort, cohort_df.loc[i].notna()]))
                        result.loc[cohort, 'Total'] = int(len(dfs[cohort].index))
                    
                    else:
                        result.loc[cohort, i] = np.nan
                        result.loc[cohort, 'n'] = int(len(dfs[cohort].index))
                
                elif i == 'APOE4':
                    
                    if 'APOE4' in list(cohort_df.index.astype(str)):
                        
                        if '2.0' not in list(cohort_df.columns.astype(str)) and '2' not in list(cohort_df.columns.astype(str)):
                            cohort_df[2.0] = np.nan
                        
                        result.loc[cohort, i] = round(100 * sum([val for val in cohort_df.loc[i, categorical[i]]]) / 
                                                     sum([val for val in cohort_df.loc[i].dropna()]), 1)
                    
                    else:
                        result.loc[cohort, i] = np.nan
                
                elif i == 'Sex':
                    
                    if (i in cohort_df.index) & ("Female" in cohort_df.columns):
                        result.loc[cohort, i] = round(100 * sum([val for val in cohort_df.loc[i, categorical[i]]]) 
                                                      / sum([val for val in cohort_df.loc[i].dropna()]), 1)
                    else:
                        result.loc[cohort, i] = 0
    
                    
    result.rename(columns={"Sex": "Female %", "APOE4": "APOE4 %"}, inplace=True)
              
    return result

In [5]:
def num_stat_df(dfs, result_df):
    """Calculating std and mean and storing it in the result dataframe"""
    
    column_names = ['Age', 'CDR', 'Education', 'MMSE', 'CDRSB', 'Hippocampus', 'A-beta', 'Ttau', 'Ptau']
    
    for df in dfs:
        dataset = dfs[df]
        calc_dict = dict()
        
        for col in column_names:
            
            if (col in dataset.columns) and (dataset[col].notna().any()):
                df_std = round(np.nanstd(dataset[col]), 1)
                df_mean = round(np.nanmean(dataset[col]), 1)
                dict_value = str(df_mean) + ' (' + str(df_std) + ')'
                calc_dict[col] = dict_value
                
            else:
                calc_dict[col] = np.nan
   
        for key in calc_dict:
            result_df.loc[df, key] = calc_dict[key]
        
    return result_df

## Make an empty dataframe to fill in with the results

In [6]:
results = pd.DataFrame(index = all_cohorts.keys(), columns = [col for col in all_cohorts['AIBL'].columns])
results.index.name = 'Name of Dataset'

for i in ['CU', 'MCI', 'AD', 'Total']:
    results[i] = np.nan

cat_stat_df(all_cohorts, results)
num_stat_df(all_cohorts, results)

results.drop(columns=['Diagnosis', 'Visit', 'Race', 'Months'], inplace=True)
results

,Age,Female %,Education,APOE4 %,CDR,MMSE,CDRSB,Hippocampus,Ttau,Ptau,A-beta,CU,MCI,AD,Total,n
Name of Dataset,,,,,,,,,,,,,,,,
A4,71.6 (4.9),57.7,16.4 (3.0),34.3,0.0 (0.1),28.6 (1.6),0.1 (0.4),6.7 (0.8),NaN,NaN,NaN,6945.0,0.0,0.0,6945.0,6945.0
ABVIB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ADNI,72.8 (6.7),55.7,16.5 (2.6),30.3,0.0 (0.0),29.1 (1.1),0.0 (0.1),7396.2 (908.5),237.5 (90.5),21.8 (9.2),1207.8 (443.5),813.0,0.0,0.0,813.0,813.0
AIBL,71.0 (7.2),57.4,12.3 (3.0),27.8,0.1 (0.2),28.7 (1.6),0.1 (0.5),2.9 (0.3),NaN,NaN,NaN,803.0,0.0,0.0,803.0,803.0
ANM,74.5 (6.4),59.4,12.3 (4.3),25.3,0.1 (0.2),28.7 (2.1),0.2 (1.2),7039.9 (874.8),NaN,NaN,NaN,793.0,0.0,0.0,793.0,793.0
ARWIBO,51.6 (15.4),60.8,10.8 (4.5),19.3,0.0 (0.1),28.8 (1.4),NaN,7907.7 (1036.1),555.5 (0.0),94.9 (0.0),630.7 (0.0),1476.0,0.0,0.0,1476.0,1476.0
DOD-ADNI,69.3 (4.6),1.1,15.1 (2.4),27.6,0.1 (0.2),28.6 (1.4),0.3 (0.6),7805.1 (896.9),212.0 (78.8),18.6 (7.8),1207.1 (484.4),181.0,0.0,0.0,181.0,181.0
EDSD,69.0 (6.0),51.9,13.2 (3.6),32.4,NaN,28.8 (1.2),NaN,7695.5 (1035.0),NaN,NaN,NaN,183.0,0.0,0.0,183.0,183.0
EMIF,65.0 (7.7),44.3,13.6 (3.6),NaN,0.0 (0.1),29.0 (1.0),NaN,7679.4 (822.8),233.5 (206.7),45.7 (22.9),665.9 (259.0),366.0,0.0,0.0,366.0,366.0


## Final table 

In [7]:
results[['n', 'Total', 'CU', 'MCI', 'AD', 'Female %', 'Age', 'Education', 'MMSE', 'CDR', 'CDRSB', 'APOE4 %', 'Hippocampus']]

,n,Total,CU,MCI,AD,Female %,Age,Education,MMSE,CDR,CDRSB,APOE4 %,Hippocampus
Name of Dataset,,,,,,,,,,,,,
A4,6945.0,6945.0,6945.0,0.0,0.0,57.7,71.6 (4.9),16.4 (3.0),28.6 (1.6),0.0 (0.1),0.1 (0.4),34.3,6.7 (0.8)
ABVIB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ADNI,813.0,813.0,813.0,0.0,0.0,55.7,72.8 (6.7),16.5 (2.6),29.1 (1.1),0.0 (0.0),0.0 (0.1),30.3,7396.2 (908.5)
AIBL,803.0,803.0,803.0,0.0,0.0,57.4,71.0 (7.2),12.3 (3.0),28.7 (1.6),0.1 (0.2),0.1 (0.5),27.8,2.9 (0.3)
ANM,793.0,793.0,793.0,0.0,0.0,59.4,74.5 (6.4),12.3 (4.3),28.7 (2.1),0.1 (0.2),0.2 (1.2),25.3,7039.9 (874.8)
ARWIBO,1476.0,1476.0,1476.0,0.0,0.0,60.8,51.6 (15.4),10.8 (4.5),28.8 (1.4),0.0 (0.1),NaN,19.3,7907.7 (1036.1)
DOD-ADNI,181.0,181.0,181.0,0.0,0.0,1.1,69.3 (4.6),15.1 (2.4),28.6 (1.4),0.1 (0.2),0.3 (0.6),27.6,7805.1 (896.9)
EDSD,183.0,183.0,183.0,0.0,0.0,51.9,69.0 (6.0),13.2 (3.6),28.8 (1.2),NaN,NaN,32.4,7695.5 (1035.0)
EMIF,366.0,366.0,366.0,0.0,0.0,44.3,65.0 (7.7),13.6 (3.6),29.0 (1.0),0.0 (0.1),NaN,NaN,7679.4 (822.8)
